# Setup

Setup by loading relevant modules and iPython extensions.

In [4]:
# Core 
import datetime
import os
import glob

# Analysis 
import xarray as xr
import numpy as np
import pyproj as pp
import scipy as sp
import transect_analysis as ta

# Plotting 
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.ticker as mticker
from matplotlib.animation import FuncAnimation
import matplotlib.colors as colors

# Debugging 
import pdb
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


Define transects.

In [3]:
# Specify start and end coords on coast.
# Choose order so that (transect_axis, coastline_axis) forms a right hand coordinate system
lon0 = 134.5293 
lat0 = -12.4715
coast_lon1 = 133.3290
coast_lat1 = -12.1468

trans_lon0, trans_lat0, trans_lon1, trans_lat1, n_points, n_trans, coast_distances, tran_distances = ta.define_transects(
    lon0, lat0, coast_lon1, coast_lat1, 453300
)

# Plot Transect and Mean Wind Map

Plot map of transects.

In [ ]:
# Load WRF December mean data
U_mean = xr.open_dataset('/g/data/w40/esh563/goulburn_NT/means/U_goulburn_12.nc')
V_mean = xr.open_dataset('/g/data/w40/esh563/goulburn_NT/means/V_goulburn_12.nc')
PRCP_mean = xr.open_dataset('/g/data/w40/esh563/goulburn_NT/means/prcp_goulburn_12.nc')

# Calcualate speed
speed = np.sqrt(U_mean.U ** 2 + V_mean.V ** 2)

# Define maximum level to plot
max_level = 8000

# Initialise fonts
rcParams.update({'font.family' : 'serif'})
rcParams.update({'font.serif': 'Liberation Serif'})
rcParams.update({'mathtext.fontset' : 'dejavuserif'}) 
rcParams.update({'font.size': 12})

# Initialise figure
plt.ioff()
plt.close('all')
fig = plt.figure(figsize=(5,5))

# Define map boundaries
lon_min = 133.01
lon_max = 135.99
lat_min = -12.45
lat_max = -8

# Set up cartopy
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent(
    [lon_min, lon_max, lat_min, lat_max], 
    crs=ccrs.PlateCarree()
)
ax.coastlines(resolution='50m')

# Draw speed shading.
speed_max = np.ceil(np.amax(speed.sel(level = slice(0,max_level)).values)*2)/2

levels = np.arange(0, speed_max+.5, 0.5)

speed_contour = ax.contourf(
    speed.longitude, speed.latitude, speed.isel(level = 0), levels=levels,
    vmin=0, vmax=speed_max, cmap='Reds', transform=ccrs.PlateCarree()
)
cbar_w = plt.colorbar(speed_contour, orientation='vertical')
cbar_w.set_label('Wind Speed [m/s]')

# Draw precipitation contours
contour_prcp = ax.contour(
    PRCP_mean.longitude,
    PRCP_mean.latitude,
    PRCP_mean.RAINNC*1000,
    levels = np.array([2 ** -2, 2 ** -1.5]),
    colors = ['royalblue', 'blue'],
    linestyles = ['dashed', 'solid'],
    linewidths = 0.75,
)

contour_prcp.collections[0].set_dashes([(0, (5.0, 2.0))])
labels = ['$2^{-2}$ mm/h', '$2^{-1.5}$ mm/h']
for i in range(len(labels)):
    contour_prcp.collections[i].set_label(labels[i])
plt.legend(loc='upper left')

# Draw arrows every half degree lon and lat
arrow_step = 0.5

U_quiver = U_mean.U.sel(
    latitude = np.arange(lat_min + arrow_step/2, lat_max + arrow_step/2, arrow_step), 
    longitude = np.arange(lon_min + arrow_step/2, lon_max + arrow_step/2, arrow_step),
    method = 'nearest'
)
V_quiver = V_mean.V.sel(
    latitude = np.arange(lat_min, lat_max, arrow_step), 
    longitude = np.arange(lon_min, lon_max, arrow_step),
    method = 'nearest'
)

ax.quiver(
    U_quiver.longitude, 
    U_quiver.latitude, 
    U_quiver.isel(level = 0), V_quiver.isel(level = 0), 
    units='xy', angles='xy', transform=ccrs.PlateCarree(),
    scale=speed_max/arrow_step
)

# Draw grid
grid = ax.gridlines(
    crs=ccrs.PlateCarree(), draw_labels=True,
    linewidth=1, color='gray', alpha=0.4, linestyle='--',
)

grid.xlabels_top = False
grid.ylabels_right = False

grid.xlocator = mticker.FixedLocator(
    np.arange(np.floor(lon_min), np.ceil(lon_max)+1, 1)
)
grid.ylocator = mticker.FixedLocator(
    np.arange(np.floor(lat_min), np.ceil(lat_max)+1, 1)
)

grid.xformatter = LONGITUDE_FORMATTER
grid.yformatter = LATITUDE_FORMATTER

# Draw transects
for k in np.arange(0,np.size(trans_lon0),3):
    ax.plot(
            [trans_lon0[k], trans_lon1[k]], [trans_lat0[k], trans_lat1[k]],
            transform=ccrs.PlateCarree(),
            color='black', linewidth=.75
    )
    
# Make labels
plt.title('December Mean Wind [m/s] \n 2004 to 2014  \n {:.3f} km'.format(speed.level.values[0]/1000))
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Create directory for saving figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")         

directory = '/g/data/w40/esh563/goulburn_NT/mean_wind_' + dt

os.mkdir(directory)

# Define animation update function
def update(i):
    
    print('Drawing frame {}.'.format(str(i).zfill(2)),  end='\r')

    # Update contour
    speed_contour = ax.contourf(
        speed.longitude, speed.latitude, speed.isel(level = i), levels=levels,
        vmin=0, vmax=speed_max, cmap='Reds', transform=ccrs.PlateCarree()
    )

    # Update quiver
    ax.quiver(
        U_quiver.longitude, 
        U_quiver.latitude, 
        U_quiver.isel(level = i), V_quiver.isel(level = i), 
        units='xy', angles='xy', transform=ccrs.PlateCarree(),
        scale=speed_max/arrow_step
    )

    # Update title
    plt.title('Mean Wind December 2012 [m/s] \n' + '{:.3f} km'.format(speed.level[i].values/1000))
        
    # Save frame
    plt.savefig(
        directory + '/{}.png'.format(str(int(speed.level.values[i])))
    )

    return speed_contour, ax

# Create and save animation
anim = FuncAnimation(
    fig, update, interval=200,
    frames=np.arange(0, np.where(speed.level.values < max_level)[0][-1] + 1)
)

anim.save(directory + '/mean_wind.gif', dpi=80, writer='imagemagick')

# Plot Cross Section and Hovmoller Diagrams

Load datsets.

In [52]:
proj_tran_mean = xr.open_dataset('/g/data/w40/esh563/goulburn_NT/transect_means/wind_proj_goulburn_12.nc')
W_tran_mean = xr.open_dataset('/g/data/w40/esh563/goulburn_NT/transect_means/W_goulburn_12.nc')
PRCP_tran_mean = xr.open_dataset('/g/data/w40/esh563/goulburn_NT/transect_means/PRCP_goulburn_12.nc')

Define LST vectors.

In [53]:
# Define local solar time based on mean of transect lon values
lon_mean = np.mean(np.concatenate((trans_lon0, trans_lon1)))
LST = (proj_tran_mean.hour.values + (lon_mean / 360) * 24) % 24
LST_h = np.floor(LST)
LST_m = np.round((LST % 1) * 60)

## Cross Section

Plot cross section.

In [191]:
print('Animating velocity cross section.')

# Turn off interacting plotting 
plt.ioff()

# Initialise fonts
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Liberation Serif'})
rcParams.update({'font.size': 12})

fig, ax = plt.subplots(figsize=(8, 8))

# Plot location of coastline
plt.plot(
    [0, 0], [proj_tran_mean.level[0], proj_tran_mean.level[57]],
    '--', color='grey'
)

# Plot speeds
speed = np.sqrt(proj_tran_mean.wind_proj ** 2 + W_tran_mean.W ** 2)

speed_max=np.ceil(np.amax(np.abs(speed[:,0:58,:].values))*4)/4
levels=np.arange(0, speed_max+0.25, 0.25)

contour_plot = ax.contourf(
    proj_tran_mean.transect_axis, proj_tran_mean.level[0:58], 
    speed.values[0,0:58,:],
    levels, vmin=0, vmax=speed_max, cmap='Reds')

cbar=plt.colorbar(contour_plot)
cbar.set_label('Perturbation Velocity [m/s]')

# Draw arrows every 50 km horizontal, 1 km height
arrow_step_v = 1000
arrow_step_h = 50000

heights = np.arange(
    proj_tran_mean.level[0]+arrow_step_v/2, proj_tran_mean.level[58], arrow_step_v
)
distances = np.arange(
    proj_tran_mean.transect_axis[0]+arrow_step_h/2, proj_tran_mean.transect_axis[-1], arrow_step_h
)

proj_tran_quiver = proj_tran_mean.interp(transect_axis = distances).interp(level = heights)
W_tran_quiver = W_tran_mean.interp(transect_axis = distances).interp(level = heights)
 
ax.quiver(
    proj_tran_quiver.transect_axis, 
    proj_tran_quiver.level, 
    proj_tran_quiver.wind_proj.values[0,:,:], W_tran_quiver.W.values[0,:,:], 
    units='xy', angles='xy', scale=speed_max / np.sqrt(arrow_step_v ** 2 + arrow_step_h ** 2)
)

# Plot labels and title
plt.title(
    'Perturbation Velocity [m/s] \n {}:{} LST'.format(
        str(int(LST_h[0])).zfill(2), str(int(LST_m[0])).zfill(2)
    )
)
plt.xlabel('Distance [m]')
plt.ylabel('Height [m]')

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")         

directory = '/g/data/w40/esh563/goulburn_NT/cross_sect_' + dt

os.mkdir(directory)

# Define frame update function
def update(i):
    # Clear collections to correct animation issue.
    ax.collections = []
    
    print('Plotting frame ' + str(i) + '.', end='\r')
    
    # Update speeds
    contourPlot=ax.contourf(
        proj_tran_mean.transect_axis, proj_tran_mean.level[0:58], 
        speed.values[i,0:58,:],
        levels, vmin=0, vmax=speed_max, cmap='Reds'
    )

    # Update arrows
    ax.quiver(
        proj_tran_quiver.transect_axis, 
        proj_tran_quiver.level, 
        proj_tran_quiver.wind_proj.values[i,:,:], W_tran_quiver.W.values[i,:,:], 
        units='xy', angles='xy', scale=speed_max / np.sqrt(arrow_step_v ** 2 + arrow_step_h ** 2)
    )
                            
    ax.plot(
        [0, 0], [proj_tran_mean.level[0], proj_tran_mean.level[57]],
        '--', color='grey'
    )
    
    # Redraw coast line
    plt.title(
        'Perturbation Velocity [m/s] \n {}:{} LST'.format(
            str(int(LST_h[i])).zfill(2), str(int(LST_m[i])).zfill(2)
        )
    )
        
    # Save frame
    plt.savefig(directory + '/' + str(LST[i]) + '.png')

    return contourPlot, ax

# Create animation
anim = FuncAnimation(fig, update, frames=np.arange(0, 24),interval=200)
anim.save(directory + '/velocity.gif', dpi=80, writer='imagemagick')

# Close any open figures
plt.close('all')

Animating velocity cross section.


## Hovmoller

Create Hovmoller diagrams.

In [54]:
fig, ax = plt.subplots(figsize=[4.5,6])

# Initialise fonts
rcParams.update({'font.family': 'serif'})
rcParams.update({'font.serif': 'Liberation Serif'})
rcParams.update({'mathtext.fontset' : 'dejavuserif'}) 
rcParams.update({'font.size': 12})

LST_hov = (np.arange(-24,48) + (lon_mean / 360) * 24)
hov_shading = np.concatenate([proj_tran_mean.wind_proj.values[:,0,:]]*3, axis=0)
hov_contours = np.concatenate([PRCP_tran_mean.RAINNC.values]*3, axis=0) * 1000

LST_hov = LST_hov[15:48+15]
hov_shading = hov_shading[15:48+15,:]
hov_contours = hov_contours[15:48+15,:]

plt.plot([0, 0], [LST_hov[0], LST_hov[-1]], '--', color='grey')

plt.title(
    'Velocity Anomaly [m/s] \n' + '{:.3f} km'.format(proj_tran_mean.level.values[0]/1000)
)
plt.xlabel('Distance [m]')
plt.ylabel('Time [h LST]')

speed_max=np.ceil(np.amax(np.abs(proj_tran_mean.wind_proj.values[:]))*4)/4
levels=np.arange(-speed_max, speed_max+0.25, 0.25)

contour_w = ax.contourf(
    proj_tran_mean.transect_axis.values, LST_hov, hov_shading,
    cmap='RdBu_r', levels = levels,
)

contour_prcp = ax.contour(
    PRCP_tran_mean.transect_axis.values,
    LST_hov,
    hov_contours,
    levels = np.array([2**-2, 2**-1]),
    colors = ['black', 'black'],
    linestyles = ['dashed', 'solid'],
    linewidths = 0.75,
)

contour_prcp.collections[0].set_dashes([(0, (5.0, 2.0))])

labels = ['$2^{-2}$ mm/h', '$2^{-1}$ mm/h']
for i in range(len(labels)):
    contour_prcp.collections[i].set_label(labels[i])
plt.legend(loc='upper right')

plt.yticks(np.arange(0, 48, step=6))

cbar_w=plt.colorbar(contour_w, orientation='horizontal')
cbar_w.set_label('Velocity Anomaly [m/s]')

dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")         

directory = '/g/data/w40/esh563/goulburn_NT/hovmoller_' + dt

os.mkdir(directory)

def update(i):
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    ax.collections = []
    
    print('Plotting frame ' + str(i) + '.', end='\r')
    
    hov_shading = np.concatenate([proj_tran_mean.wind_proj.values[:,i,:]]*3, axis=0)
    hov_shading = hov_shading[15:48+15,:]

    contour_w = ax.contourf(
        proj_tran_mean.transect_axis.values, LST_hov, hov_shading,
        cmap='RdBu_r', levels = levels
    )
    
    contour_prcp = ax.contour(
        PRCP_tran_mean.transect_axis.values,
        LST_hov,
        hov_contours,
        levels = np.array([2**-2, 2**-1]),
        colors = ['black', 'black'],
        linestyles = ['dashed', 'solid'],
        linewidths = 0.75,
    )
    contour_prcp.collections[0].set_dashes([(0, (5.0, 2.0))])
    
    plt.yticks(np.arange(0, 48, step=6))
                            
    plt.plot([0, 0], [LST_hov[0], LST_hov[-1]], '--', color='grey')
    
    plt.title(
        'Velocity Anomaly [m/s] \n' + '{:.3f} km'.format(proj_tran_mean.level.values[i]/1000)
    )
        
    plt.savefig(
        directory + '/' + str(int(proj_tran_mean.level.values[i])) + '.png'
    )

    return contour_w, ax

anim = FuncAnimation(fig, update, frames=np.arange(0, proj_tran_mean.level.values.size), interval=200)
anim.save(directory + '/hov.gif', dpi=80, writer='imagemagick')

plt.close('all')